In [ ]:

# Given number of nodes, edges, and expected motif proportions,
# construct a network with diameter ≈ ln(N),
# measure actual motif proportions, optimize via rewiring,
# and compare with expected proportions.


import networkx as nx
import numpy as np
import math
import random
import matplotlib.pyplot as plt

# Step 1: Generate Directed Random Network (Erdős–Rényi type)

def make_directed_gnp_ln_diameter(N, alpha=2.0, seed=None):
    """
    Construct directed Erdős–Rényi graph with edge probability p = alpha * log(N) / N.
    Returns a directed NetworkX DiGraph.
    """
    if seed is not None:
        np.random.seed(seed)
        random.seed(seed)
    p = max(0.0, min(1.0, alpha * math.log(max(2, N)) / N))
    G = nx.DiGraph()
    G.add_nodes_from(range(N))
    for u in range(N):
        for v in range(N):
            if u != v and random.random() < p:
                G.add_edge(u, v)
    return G

# Step 2: Safe diameter computation

def safe_diameter(G):
    """Compute diameter using the largest connected component."""
    Gu = G.to_undirected()
    comps = list(nx.connected_components(Gu))
    largest = max(comps, key=len)
    sub = Gu.subgraph(largest)
    return nx.diameter(sub)


# Step 3: Triadic census (motif counts & proportions)

def triad_proportions(G):
    counts = nx.triadic_census(G)
    total_triples = math.comb(G.number_of_nodes(), 3)
    proportions = {k: v / total_triples for k, v in counts.items()}
    return counts, proportions


# Step 4: Objective distance (L2 norm between motifs)

def objective_distance(observed_props, target_props, triad_keys=None, weight_diameter=0.0, curr_diam=None, target_diam=None):
    if triad_keys is None:
        triad_keys = list(observed_props.keys())
    s = sum((observed_props.get(k, 0.0) - target_props.get(k, 0.0)) ** 2 for k in triad_keys)
    dist = math.sqrt(s)
    if weight_diameter and curr_diam is not None and target_diam is not None:
        dist += weight_diameter * abs(curr_diam - target_diam)
    return dist


# Step 5: Rewiring preserving edge count (for optimization)

def propose_rewire_preserve_edgecount(G):
    edges = list(G.edges())
    if len(edges) < 2:
        return None
    for _ in range(100):
        a, b = random.choice(edges)
        c, d = random.choice(edges)
        if a == c or b == d or a == d or c == b:
            continue
        if not G.has_edge(a, d) and not G.has_edge(c, b):
            return (a, b, c, d)
    return None

def apply_rewire(G, a, b, c, d):
    G.remove_edge(a, b)
    G.remove_edge(c, d)
    G.add_edge(a, d)
    G.add_edge(c, b)


# Step 6: Simulated annealing optimization

def try_optimize_motifs(G_init, target_props, iterations=2000, temp0=0.02, weight_diameter=0.02, verbose=True):
    G = G_init.copy()
    N = G.number_of_nodes()
    target_diam = math.log(max(2, N))
    counts, props = triad_proportions(G)
    curr_diam = safe_diameter(G)
    curr_obj = objective_distance(props, target_props, weight_diameter=weight_diameter, curr_diam=curr_diam, target_diam=target_diam)

    history = [(0, curr_obj, curr_diam)]
    if verbose:
        print(f"Init diameter: {curr_diam:.3f}, Objective: {curr_obj:.6f}")

    for it in range(1, iterations + 1):
        proposal = propose_rewire_preserve_edgecount(G)
        if proposal is None:
            continue
        a, b, c, d = proposal
        apply_rewire(G, a, b, c, d)

        _, props_new = triad_proportions(G)
        diam_new = safe_diameter(G)
        obj_new = objective_distance(props_new, target_props, weight_diameter=weight_diameter, curr_diam=diam_new, target_diam=target_diam)

        temp = temp0 * (1 - it / iterations)
        accept = obj_new <= curr_obj or (temp > 0 and math.exp(-(obj_new - curr_obj) / temp) > random.random())

        if accept:
            curr_obj = obj_new
            curr_diam = diam_new
            props = props_new
        else:
            apply_rewire(G, a, d, c, b)  # revert

        if it % (iterations // 10) == 0 and verbose:
            print(f"Iter {it}: Obj={curr_obj:.6f}, Diam={curr_diam:.3f}")
            history.append((it, curr_obj, curr_diam))
    return G, history


# Step 7: Visualization helper

def plot_motif_proportions(before, after, title="Motif Proportions Comparison"):
    keys = list(before.keys())
    before_vals = [before[k] for k in keys]
    after_vals = [after[k] for k in keys]

    x = np.arange(len(keys))
    plt.figure(figsize=(12, 6))
    plt.bar(x - 0.2, before_vals, width=0.4, label="Before", color='skyblue')
    plt.bar(x + 0.2, after_vals, width=0.4, label="After", color='orange')
    plt.xticks(x, keys, rotation=45)
    plt.ylabel("Proportion")
    plt.title(title)
    plt.legend()
    plt.tight_layout()
    plt.show()


# Step 8: Main experiment

if __name__ == "__main__":
    N = 200
    alpha = 2.0
    G = make_directed_gnp_ln_diameter(N, alpha=alpha, seed=42)

    print("\n Network Inofrmation")
    print(f"Nodes: {G.number_of_nodes()}")
    print(f"Edges: {G.number_of_edges()}")
    init_diam = safe_diameter(G)
    print(f"Approx Diameter (ln N ≈ {math.log(N):.2f}): {init_diam:.2f}")

    counts, props = triad_proportions(G)
    print("\nInitial Motif Proportions (nonzero only):")
    for k, v in props.items():
        if v > 0:
            print(f"  {k}: {v:.6f}")

    # Example target: slightly more triangle-rich
    target_props = {k: props[k] + (0.005 if k in ['030T', '030C', '300'] else 0.0) for k in props}
    total = sum(target_props.values())
    target_props = {k: v / total for k, v in target_props.items()}

    print("\n Starting optimization")
    G_opt, hist = try_optimize_motifs(G, target_props, iterations=1000, temp0=0.02, weight_diameter=0.02)

    print("\n Final network information")
    final_counts, final_props = triad_proportions(G_opt)
    final_diam = safe_diameter(G_opt)

    print(f"Final Diameter: {final_diam:.2f}")
    print("\nFinal Motif Proportions (nonzero only):")
    for k, v in final_props.items():
        if v > 0:
            print(f"  {k}: {v:.6f}")

    # Compare distances
    def dist(a, b): return math.sqrt(sum((a.get(k, 0)-b.get(k, 0))**2 for k in a))
    print("\n comparision")
    print(f"Initial L2 distance to target: {dist(props, target_props):.6f}")
    print(f"Final   L2 distance to target: {dist(final_props, target_props):.6f}")

    # Plot comparison
    plot_motif_proportions(props, final_props, title="Motif Proportion Before vs After Optimization")



 Network Inofrmation
Nodes: 200
Edges: 2062
Approx Diameter (ln N ≈ 5.30): 3.00

Initial Motif Proportions (nonzero only):
  003: 0.726458
  012: 0.238741
  102: 0.006488
  021D: 0.006421
  021U: 0.006473
  021C: 0.012996
  111D: 0.000699
  111U: 0.000699
  030T: 0.000702
  030C: 0.000241
  201: 0.000018
  120D: 0.000013
  120U: 0.000014
  120C: 0.000037
  210: 0.000003

 Starting optimization
Init diameter: 3.000, Objective: 0.060124
Iter 100: Obj=0.060377, Diam=3.000
